In [33]:
import boto3 # Python integration with AWS.
import pandas as pd # Read csv.
from io import StringIO, BytesIO # StringIO to read csv on AWS. BytesIO to save as Parquet file to S3.
from datetime import datetime, timedelta # Date functions.

In [34]:
# Read meta_file.csv 

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    # Read csvs with StringIO (needed for AWS) and Pandas
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

meta_key = 'meta_file.csv'
bucket_name_trg = 'manoetl123'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(bucket_name_trg)

df_meta = read_csv_to_df(bucket_trg, meta_key)

In [35]:
df_meta

,source_date,datetime_of_processing
0,2021-11-25,2021-11-25 12:33:23
1,2021-11-26,2021-11-26 12:30:21


In [58]:
# Return date list
arg_date = '2021-11-25'
today_str = '2021-11-27'
scr_format = '%Y-%m-%d'
# Convert to date type and use previous day
min_date = datetime.strptime(arg_date, scr_format).date() - timedelta(days=1)
today = datetime.strptime(today_str, scr_format).date()
# Return dates as objects in list.
return_date_list = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]

In [59]:
# Dates list from meta file
src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)

In [60]:
# Min date
dates_missing = set(return_date_list[1:]) - src_dates
if dates_missing:
    min_date = min(set(return_date_list[1:]) - src_dates) - timedelta(days=1)
    return_dates = [date.strftime(scr_format) for date in return_date_list if date >= min_date]
    return_min_date = arg_date
else:
    return_dates = []
    return_min_date = datetime(2200, 1, 1).date()

In [61]:
return_dates

['2021-11-26', '2021-11-27']

In [57]:
    return_min_date

datetime.date(2200, 1, 1)

## Functions

In [18]:
# Adapter Layer Functions

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    # Read csvs with StringIO (needed for AWS) and Pandas
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [19]:
# Application Layer Functions, ETL

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    # DF with filtered columns.
    df = df.loc[:, columns]
    # Filter out empty lines.
    df.dropna(inplace=True)
    # Get opening price per ISIN and day
    # Create new column with opening price.
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    # Get closing price per ISIN and day
    # Create new column with closing price.
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    # Aggregation
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    # Percent Change Prev Closing
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    # Remove column and update df
    df.drop(columns=['prev_closing_price'], inplace=True)
    # Round everything.
    df = df.round(decimals=2)
    # Filter by date, don't show previous date that was used for calculation.
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    # Write to S3.
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

# Combine all three ETL functions.
def etl_report1(scr_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df = extract(scr_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True

In [20]:
# Application layer - not core

# Restructure get_objects
def return_date_list(bucket, arg_date, scr_format):
    # Convert to date type and use previous day
    min_date = datetime.strptime(arg_date, scr_format).date() - timedelta(days=1)
    today = datetime.today().date()
    # Return date list in string format.
    return_date_list = [(min_date + timedelta(days=x)).strftime(scr_format) for x in range(0, (today-min_date).days + 1)]
    return return_date_list

In [21]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    # Argument Date for start of report
    arg_date = '2022-11-24'
    # Moving all arguments and parameters to the beggining of the code
    scr_format = '%Y-%m-%d'
    scr_bucket = 'xetra-1234'
    trg_bucket = 'manoetl123'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    
    # Init
    # Variables for data on AWS S3.
    s3 = boto3.resource('s3')
    bucket_scr = s3.Bucket(scr_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    date_list = return_date_list(bucket_scr, arg_date, scr_format)
    etl_report1(bucket_scr, bucket_trg, date_list, columns, arg_date, trg_key, trg_format)

In [22]:
# run

main()

## Read the uploaded file

In [23]:
trg_bucket = 'manoetl123'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)

# Read the uploaded file
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report_20221122_222801.parquet
xetra_daily_report_20221122_233442.parquet
xetra_daily_report_20221123_183552.parquet
xetra_daily_report_20221124_210807.parquet


In [26]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20221124_210807.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [27]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-11-24,36.05,36.35,35.65,37.05,1838,-4.22
1,AT00000FACC2,2022-11-24,7.94,7.80,7.73,7.94,2785,-6.70
2,AT0000606306,2022-11-24,19.15,15.92,15.92,19.49,101681,-25.68
3,AT0000609607,2022-11-24,12.00,11.36,11.26,12.00,824,-7.04
4,AT0000644505,2022-11-24,96.30,94.60,93.70,97.20,656,-4.25
...,...,...,...,...,...,...,...,...
3125,XS2284324667,2022-11-24,40.03,40.31,39.99,40.90,9797,4.81
3126,XS2314659447,2022-11-24,8.76,8.94,8.76,9.02,52,2.81
3127,XS2314660700,2022-11-24,22.76,23.60,22.76,23.60,150,8.13
3128,XS2376095068,2022-11-24,31.63,31.57,31.57,31.63,3,-7.40
